In [ ]:
import pandas as pd

In [ ]:
ls -lah ../ref

In [ ]:
exp_df = pd.read_csv('../ref/2022-01-24_NextSeq.csv', header=0, index_col=False, nrows=16, usecols=[1,2,4])
exp_df['Sample_id'] = exp_df.apply(lambda x: x['Sample_id'].split('_')[1], axis=1)
exp_df['Condition'] = exp_df.apply(lambda x: x['Sample'].strip('0123456789-'), axis=1)
exp_df

## Count matrix

In [ ]:
def count_matrix(exp_df, res_dir='../results'):
    df = pd.DataFrame()
    for i,row in enumerate(exp_df.iterrows()):
        _, sample = row
        filename = '{res_dir}/{sample}/{sample}.htseq.counts'.format(
                    res_dir=res_dir, sample=sample['Sample'])
        ct = pd.read_csv(filename, header=None, names=['gene', 'counts'],
                        sep='\t', skipfooter=5, engine='python')
        if df.empty:
            df = ct.copy()
            df.rename(columns={'counts': f'{sample["Sample"]}'}, inplace=True)
        else:
            df[f'{sample["Sample"]}'] = ct['counts']
    df.set_index('gene', inplace=True)
    df.index.names = [None]
    print(df)
    return df

In [ ]:
from pathlib import Path
from plotnine import *

# Group by tissue
exp_df['Sample'] = exp_df['Sample_id']
ctrl = exp_df[exp_df['Condition']=='MDX-GFP']
for name,grouped in exp_df.groupby('Condition'):
    if name != 'MDX-GFP':
        df = count_matrix(pd.concat([ctrl,grouped]))
        df.to_csv(f'../results/count_matrix_{name}.csv', index_label=False)
        print(df.sum(axis=0))
        print(ggplot(df.melt(var_name='sample'), aes(x='value'))
                + geom_density(alpha=0.4)
                + scale_x_log10()
                + facet_wrap('sample')
                    )

In [ ]:
df = count_matrix(exp_df.loc[10:,:])
df.to_csv('../results/count_matrix_DMDvsDual.csv', index_label=False)
print(df.sum(axis=0))
print(ggplot(df.melt(var_name='sample'), aes(x='value'))
        + geom_density(alpha=0.4)
        + scale_x_log10()
        + facet_wrap('sample')
            )

In [ ]:
ls -lah ../results/